In [ ]:
import time, sys, timeit
from IPython.display import clear_output
import numpy as np
import itertools as it
import math

### For reproducibility
np.random.seed(2022)

In [ ]:
### Initialisation
N = 50
C = math.comb(N,2)
z0 = np.arange(1,N+1)
beta = 1

def gen_idx_pairs():
    return np.array(list(it.combinations(z0,2))) - [1,1]

def gen_idx_pairs_2():
    return np.vstack(np.triu_indices(N, k=1)).T 

def gen_idx_pairs_3():
    return np.fromiter(it.chain.from_iterable(it.combinations(np.arange(1,N+1), 2)), np.int64)

number = 10000
fct_to_test = ["gen_idx_pairs()", "gen_idx_pairs_2()", "gen_idx_pairs_3()"]
result_list = []
for fct in fct_to_test:
    result = timeit.timeit(stmt=fct, globals=globals(), number=number)
    result_list += [result]
    print(f"Average execution time for {fct} is {result / number} seconds")
print(f"Best function is : {fct_to_test[np.argmin(np.array(result_list))]}")

In [ ]:
idx_pairs = gen_idx_pairs_2()
def compute_col_diff():
    return np.array([j-i for (i,j) in idx_pairs])

def compute_col_diff_2():
    return np.apply_along_axis(lambda pair : pair[1] - pair[0], axis=1, arr=idx_pairs)

def compute_col_diff_3():
    return (idx_pairs[:, 1] - idx_pairs[:, 0]).flatten()
    

number = 1000
fct_to_test = ["compute_col_diff()", "compute_col_diff_2()", "compute_col_diff_3()"]
result_list = []
for fct in fct_to_test:
    result = timeit.timeit(stmt=fct, globals=globals(), number=number)
    result_list += [result]
    print(f"Average execution time for {fct} is {result / number} seconds")
print(f"Best function is : {fct_to_test[np.argmin(np.array(result_list))]}")

In [ ]:
col_diff = compute_col_diff_3()
z = z0.copy()
def compute_row_diff():
    return np.array([abs(z[j]-z[i]) for (i,j) in idx_pairs])

def compute_row_diff_2():
    return np.abs(z[idx_pairs[:, 1]]-z[idx_pairs[:, 0]])
    
number = 1000
fct_to_test = ["compute_row_diff()", "compute_row_diff_2()"]
result_list = []
for fct in fct_to_test:
    result = timeit.timeit(stmt=fct, globals=globals(), number=number)
    result_list += [result]
    print(f"Average execution time for {fct} is {result / number} seconds")
print(f"Best function is : {fct_to_test[np.argmin(np.array(result_list))]}")

In [ ]:
range_ = np.arange(0,N)
def compute_threats(z = z, i = 5):
    return np.sum([abs(k-i)==abs(z[k]-z[i]) for k in range(N) if k != i])

def compute_threats_2(z = z, i = 5):
    tmp_range = np.delete(range_, i)
    return np.sum(np.abs(tmp_range - i) == np.abs(z[tmp_range] - z[i]))

def compute_threats_3(z = z, i = 5):
    tmp_range = np.array([k for k in range(N) if k != i])
    return np.sum(np.abs(tmp_range - i) == np.abs(z[tmp_range] - z[i]))

number = 10000
fct_to_test = ["compute_threats()", "compute_threats_2()", "compute_threats_3()"]
result_list = []
for fct in fct_to_test:
    result = timeit.timeit(stmt=fct, globals=globals(), number=number)
    result_list += [result]
    print(f"Average execution time for {fct} is {result / number} seconds")
print(f"Best function is : {fct_to_test[np.argmin(np.array(result_list))]}")

In [ ]:
def swap(z, i, j):
    z[[i, j]] = z[j], z[i]
    return z

def threats(z, i):
    tmp_range = np.delete(range_, i)
    return np.sum(np.abs(tmp_range - i) == np.abs(z[tmp_range] - z[i]))


def compute_loss_diff(z = z, i = 5, j = 10):
    old = threats(z,i) + threats(z,j)
    y = swap(z.copy(), i, j)
    new = threats(y,i) + threats(y,j)
    return new - old

def compute_loss_diff_2(z = z, i = 5, j = 10):
    old = threats(z,i) + threats(z,j)
    z = swap(z, i, j)
    new = threats(z,i) + threats(z,j)
    z = swap(z, i, j)
    return new - old

number = 10000
fct_to_test = ["compute_loss_diff()", "compute_loss_diff_2()"]
result_list = []
for fct in fct_to_test:
    result = timeit.timeit(stmt=fct, globals=globals(), number=number)
    result_list += [result]
    print(f"Average execution time for {fct} is {result / number} seconds")
print(f"Best function is : {fct_to_test[np.argmin(np.array(result_list))]}")

It looks like compute_loss_diff_2 and compute_loss_diff have similar average time but in our algo we observed that compute_loss_diff_2 was taking much less time.

In [ ]:
z = z0.copy()

def loss(z):
    row_diff = np.abs(z[idx_pairs[:, 1]]-z[idx_pairs[:, 0]])
    loss = np.sum(col_diff==row_diff)
    return loss

def loss_diff(z, i, j):
    old = threats(z,i) + threats(z,j)
    y = swap(z, i, j)
    new = threats(y,i) + threats(y,j)
    z = swap(y, i, j)
    return new - old

I = np.append(idx_pairs, [[0, 0]],axis=0)

def compute_P(z= z, t = 10, I = I):
    return np.array([min(1, np.exp(-np.log(t**2)*loss_diff(z,i,j))) / C for (i,j) in I])

range_ = np.reshape(np.arange(0,N), (-1, 1))

def swap_array(z, i_array, j_array):
    i_array = np.reshape(i_array, (-1, 1))
    j_array = np.reshape(j_array, (-1, 1))
    tmp_Zi = np.take_along_axis(z, i_array, axis=1)
    tmp_Zj = np.take_along_axis(z, j_array, axis=1)
    np.put_along_axis(z, j_array, tmp_Zi, axis = 1)
    np.put_along_axis(z, i_array, tmp_Zj, axis = 1)
    return z

def threats_array(z, i_array):
    tmp_range = np.repeat(range_, len(i_array), axis=1).T
    tmp_range = tmp_range[np.arange(tmp_range.shape[1]) != i_array[:,None]].reshape(tmp_range.shape[0],-1)
    i_array = np.reshape(i_array, (-1, 1))
    return np.sum(np.abs(tmp_range - i_array) == np.abs(np.take_along_axis(z, tmp_range, axis=1) - np.take_along_axis(z, i_array, axis=1)), axis = 1)
    
def loss_diff_array(z, i_array, j_array):
    old = threats_array(z, i_array) + threats_array(z, j_array)
    y = swap_array(z, i_array, j_array)
    new = threats_array(y, i_array) + threats_array(y, j_array)
    z = swap_array(y, i_array, j_array)
    return new - old
    
def compute_P_2(z= z, t = 10, I = I): # Vectorialized
    z_tmp = np.reshape(z, (1, -1))
    z_tmp = np.repeat(z_tmp, I.shape[0], axis=0)
    return np.minimum(1, np.exp(-np.log(t**2)*loss_diff_array(z_tmp, I[:, 0], I[:, 1]))) / C

number = 100
fct_to_test = ["compute_P_2()", "compute_P()"]
result_list = []
for fct in fct_to_test:
    result = timeit.timeit(stmt=fct, globals=globals(), number=number)
    result_list += [result]
    print(f"Average execution time for {fct} is {result / number} seconds")
print(f"Best function is : {fct_to_test[np.argmin(np.array(result_list))]}")


In [ ]:
np.random.seed(2022)

### Initialisation
N = 50
C = math.comb(N,2)
z0 = np.arange(1,N+1)
beta = 1

idx_pairs = np.vstack(np.triu_indices(N, k=1)).T 
col_diff = (idx_pairs[:, 1] - idx_pairs[:, 0]).flatten()

def swap(z, i, j):
    z[[i, j]] = z[j], z[i]
    return z

def loss(z):
    row_diff = np.abs(z[idx_pairs[:, 1]]-z[idx_pairs[:, 0]])
    loss = np.sum(col_diff==row_diff)
    return loss
    
def threats(z, i):
    tmp_range = np.delete(range_, i)
    return np.sum(np.abs(tmp_range - i) == np.abs(z[tmp_range] - z[i]))

def loss_diff(z, i, j):
    old = threats(z,i) + threats(z,j)
    y = swap(z, i, j)
    new = threats(y,i) + threats(y,j)
    z = swap(y, i, j)
    return new - old

### Run search
MAX_ITERS=100000
z = z0.copy()
P = [0]
I = np.append(idx_pairs, [[0, 0]],axis=0)

for t in range(1, MAX_ITERS):
    # Calculate loss
    l = loss(z)
    
    # Print current loss
    clear_output(wait = True)
    print("t =", t, "| conflicts =", l)
    print(min(P), max(P))
    
    # If a solution is found, exit.
    if l == 0:
        break
        
    # Otherwise, evolve z.
    else:
        # Outgoing arrows. This step is O(N^3), which is better than O(N!)
        # P_2 = np.array([min(1, np.exp(-np.log(t**2)*loss_diff(z,i,j))) / C for (i,j) in I])
        P = compute_P_2(z, t, I)
        # assert np.all(P == P_2)

        # Self loop
        P[-1] = max(0, 1 - sum(P[:-1]))

        # Choose a good z
        i, j = I[np.random.choice(C+1, size=1, p=P)][0]
        z = swap(z, i, j)

if (loss(z) == 0):    
    print("Here's a valid solution: ", z, "\nFound after ", t, " steps. (beta = ", np.log(t), ")")

Time taken : about 30 sec.